In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

import torch
import torch.nn as nn
import torch.optim as optim

from datetime import datetime

In [ ]:
'''Domain Setting: Data generation'''
class Set(object):
    def __init__(self,lb_I,ub_I,lb_x,ub_x):
        self.lb_I = lb_I
        self.ub_I = ub_I
        self.lb_x = lb_x
        self.ub_x = ub_x
    
    def generate_data(self,N,M):
        t = torch.linspace(self.lb_I,self.ub_I,M,requires_grad = True)
        x = torch.linspace(self.lb_x,self.ub_x,N,requires_grad = True)
        
        T,X = torch.meshgrid(t,x,indexing='ij')
        tx = torch.stack((T.flatten(),X.flatten()),dim=1)
        
        return tx
    
    def initial_data(self,x):
        x0 = torch.stack((torch.zeros(x.size()[0]),x),dim=1)
        return x0.requires_grad_()
        

'''DNN Setting: Activation function and DNN Class'''
        
class Sin(torch.nn.Module):
    def __init__(self):
        super(Sin, self).__init__()
        return
    def forward(self, x):
        return torch.sin(x)

class NeuralNetwork(nn.Module):
    def __init__(self, dim_input, dim_output, n_layers, wide, activation, conv = False, Drop = 0, init = False):
        super().__init__()
        self.conv = conv
        self.Drop = Drop
        self.init = init
        
        if conv:
            self.conv1 = nn.Conv1d(in_channels=2, out_channels=8, kernel_size=1)
            self.conv2 = nn.Conv1d(8, 16, kernel_size=1)
            
            self.inner_layers = nn.ModuleList([nn.Linear(16 if n == 0 else wide, wide)
                                               for n in range(n_layers - 1)])
        else:
            self.inner_layers = nn.ModuleList([nn.Linear(dim_input if n == 0 else wide, wide)
                                               for n in range(n_layers - 1)])
        self.last_layer = nn.Linear(wide, dim_output)
        
        self.activation = activation
        self.dropout = nn.Dropout(Drop)
        if init:
            self._initialize_weights()

    def _initialize_weights(self):
        if self.conv:
            nn.init.kaiming_uniform_(self.conv1.weight)
            nn.init.constant_(self.conv1.bias, 0)
            nn.init.kaiming_uniform_(self.conv2.weight)
            nn.init.constant_(self.conv2.bias, 0)
        
        for layer in self.inner_layers:
            if isinstance(layer, nn.Linear):
                nn.init.kaiming_uniform_(layer.weight)
                nn.init.constant_(layer.bias, 0)       
        nn.init.kaiming_uniform_(self.last_layer.weight)
        nn.init.constant_(self.last_layer.bias, 0)

    def forward(self, input):
        if self.conv:
            input = input.unsqueeze(2)

            input = self.activation()(self.conv1(input))
            input = self.dropout(input)
            input = self.activation()(self.conv2(input))
            input = self.dropout(input)

            input = input.view(input.size(0), -1)
        
        for layer in self.inner_layers:
            input = self.activation()(layer(input))
            input = self.dropout(input)
        
        return self.last_layer(input)

In [ ]:
'''PDE Setting: Parameters and Loss'''
class PDE(object):
    def __init__(self,alpha):
        self.alpha = alpha
        self.p = 4*(alpha+1)/(alpha-1)
        self.q = alpha+1
        self.ppr = (1-1/self.p)**(-1)
        self.qpr = (1-1/self.q)**(-1)
        
    def jacobian(self, y, x):
        jac = []
        for i in range(y.size()[1]):  
            grad = torch.autograd.grad(y[:, i], x, grad_outputs=torch.ones((x.size(0),)) , create_graph=True)[0]
            jac.append(grad) 
        return torch.stack(jac, dim=1)  

    def hessian(self,y, x):
        hess = []
        jac = self.jacobian(y,x)
        for i in range(x.size(1)):  
            jac_i = jac[:,:,i]
            jac_i2 = self.jacobian(jac_i,x)
            hess.append(jac_i2)
        return torch.stack(hess, dim=2)
    
    def __call__(self,y,x):
        dy_dt = self.jacobian(y,x)[:,:,0]
        dy_dxx = self.hessian(y,x)[:,:,1,1]

        nnorm = torch.norm(y,dim=1)

        pde_real = -dy_dt[:,1]+dy_dxx[:,0] + torch.pow(nnorm,self.alpha-1) * y[:,0]
        pde_imag = dy_dt[:,0]+dy_dxx[:,1] + torch.pow(nnorm,self.alpha-1) * y[:,1]
        aux = torch.stack((pde_real,pde_imag),dim=1)
        return aux

In [ ]:
'''Exact solutions for examples'''
class Soliton(object):
    def __init__(self,nu,c):
        self.c = c
        self.nu = nu
        self.omega = nu**2/4 - c
        self.k = nu/2
    
    def initial_condition(self,x):
        x0 = torch.stack((torch.zeros(x.size()[0]),x),dim=1)
        u0 = self.__call__(x0)
        return torch.stack((u0.real,u0.imag),dim=1)
        
    def __call__(self,tx):
        t = tx[:,0]
        x = tx[:,1]
        
        return torch.exp(1j*self.k*x-1j*self.omega*t)*np.sqrt(2*self.c)/torch.cosh(np.sqrt(self.c)*(x-self.nu*t))
    

class StandingWave(object):
    def __init__(self,omega,alpha):
        self.alpha = alpha
        self.omega = omega
        
    def initial_condition(self,x):
        x0 = torch.stack((torch.zeros(x.size()[0]),x),dim=1)
        u0 = self.__call__(x0)
        return torch.stack((u0.real,u0.imag),dim=1)
    
    def __call__(self,tx):
        t = tx[:,0]
        x = tx[:,1]
        
        return torch.exp(1j*self.omega*t)*torch.pow((self.alpha+1)/2*self.omega/torch.pow(torch.cosh((self.alpha-1)/2*np.sqrt(self.omega)*x),2),1/(self.alpha-1))


class PeregrineBreather(object):
    def __init__(self):
        self.a = 0
        
    def initial_condition(self,x):
        x0 = torch.stack((torch.zeros(x.size()[0]),x),dim=1)
        u0 = self.__call__(x0)
        return torch.stack((u0.real,u0.imag),dim=1)
    
    def __call__(self,tx):
        t = tx[:,0]
        x = tx[:,1]
        
        return torch.exp(1j * t)*(1-4*(1+2*1j*t)/(1+4*torch.pow(t,2)+ 2*torch.pow(x,2)))


class KuznetsovMaBreather(object):
    def __init__(self,a):
        self.a = a
        self.alpha = (8*a*(2*a-1))**(1/2)
        self.beta = (2*(2*a-1))**(1/2)
        
    def initial_condition(self,x):
        x0 = torch.stack((torch.zeros(x.size()[0]),x),dim=1)
        u0 = self.__call__(x0)
        return torch.stack((u0.real,u0.imag),dim=1)
        
    def __call__(self,tx):
        t = tx[:,0]
        x = tx[:,1]
        
        return torch.exp(1j * t)*(1-np.sqrt(2)*self.beta*(self.beta**2*torch.cos(self.alpha*t)+1j*self.alpha*torch.sin(self.alpha*t))/(self.alpha*torch.cosh(self.beta*x)-np.sqrt(2)*self.beta*torch.cos(self.alpha*t)))
    
    
class AkhmedievBreather(object):
    def __init__(self,a):
        self.a = a
        self.beta = (8*a*(1-2*a))**(1/2)
        self.alpha = (2*(1-2*a))**(1/2)
        
    def initial_condition(self,x):
        x0 = torch.stack((torch.zeros(x.size()[0]),x),dim=1)
        u0 = self.__call__(x0)
        return torch.stack((u0.real,u0.imag),dim=1)
        
    def __call__(self,tx):
        t = tx[:,0]
        x = tx[:,1]
        
        return torch.exp(1j * t)*(1+(self.alpha**2*torch.cosh(self.beta*t)+1j*self.beta*torch.sinh(self.beta*t))/(np.sqrt(2*self.a)*torch.cos(self.alpha*x)-torch.cosh(self.beta*t)))
    
class ScaledSolution(object):
    def __init__(self,sol,mu,alpha):
        self.sol = sol
        self.mu = mu
        self.alpha = alpha
    
    def scaled_initial_condition(self,x):
        mux = x*self.mu
        solution = self.sol.initial_condition(mux)
        
        return self.mu**(2/(self.alpha-1)) * solution
    
    def scaled_sol(self,tx):
        mu2tmux = torch.zeros(tx.size())
        mu2tmux[:,0] = tx[:,0]*self.mu**2
        mu2tmux[:,1] = tx[:,1]*self.mu
        
        solution = self.sol(mu2tmux)
        
        return self.mu**(2/(self.alpha-1)) * solution

In [ ]:
'''PINNs Algorithm'''
class PINN(object):
    def __init__(self,initial_condition,pde,Set,N,Npr,M,Mpr,n_layers,wide,lr,max_iter, u_true = None, activation = Sin, optimizer = 'LBFGS', model = 1,conv = False, Drop = 0, init = False, gamma = 1):
        self.initial_condition = initial_condition
        self.pde = pde
        self.set = Set
        self.net = NeuralNetwork(2, 2, n_layers, wide, activation, conv, Drop, init)
        if optimizer == 'Adam':
            self.optimizer = torch.optim.Adam(self.net.parameters(),lr=lr)
        if optimizer == 'LBFGS':
            self.optimizer = torch.optim.LBFGS(self.net.parameters(), lr = lr, max_iter = max_iter)
        self.eta = 0
        self.max_iter = max_iter
        self.model = model
        self.gamma = gamma
        
        self.sol = u_true
        
        self.p = pde.p
        self.ppr = pde.ppr
        self.q = pde.q
        self.qpr = pde.qpr
        
        self.tx = self.set.generate_data(N,M)
        self.txpr = self.set.generate_data(Npr,Mpr)
        
        self.weights = torch.ones((M,N))
        self.weightspr = torch.ones((Mpr,Npr))
        
        self.n_test = 100
        self.m_test = 100
        self.test = Set.generate_data(self.n_test,self.m_test)
        
        self.N = N
        self.Npr = Npr
        self.M = M
        self.Mpr = Mpr
        
        self.tildeA = []
        self.A = []
        self.B = []
        self.losses = []
        
        self.LpW1q_norms = []
        self.LinftyH1_norms = []
        self.Sprime_norms = []
        self.norm_vector_Sprime = []
        self.u_true = u_true 
        
    def J_H1(self,f,x,weights):
        ff = self.pde.jacobian(f,x)
    
        normsf = torch.norm(f,dim=1)
        normsff = torch.norm(ff,dim=1)

        inside = weights*(torch.pow(normsf,2) + torch.pow(normsff,2))

        return torch.pow(torch.mean(inside),1/2)
    
    def J_inftyH1(self,g,tx,N,M,weights):
        gx = self.pde.jacobian(g,tx)[:,:,1]

        normsg = torch.norm(g,dim=1)
        normsgx = torch.norm(gx,dim=1)

        normsg_reshaped = normsg.view(M,N)
        normsgx_reshaped = normsgx.view(M,N)

        inside = weights * (torch.pow(normsg_reshaped,2)+torch.pow(normsgx_reshaped,2))
        mean1 = torch.mean(inside,dim=1)
        inside2 = torch.pow(mean1,1/2)

        return torch.max(inside2)
    
    def J_pq(self, g, p_tensor, q_tensor, N, M, weights):
        norms = torch.norm(g, dim=1)
        norms_reshaped = norms.view(M, N)
        q_expanded = q_tensor.view(-1, 1, 1)

        inside = torch.pow(norms_reshaped.unsqueeze(0), q_expanded) * weights.unsqueeze(0)
        
        mean1 = torch.mean(inside, dim=2) 
        
        q_expanded = q_tensor.view(-1, 1) 
        p_expanded = p_tensor.view(-1, 1)     
        inside2 = torch.pow(mean1, p_expanded / q_expanded)  

        mean2 = torch.mean(inside2, dim=1)
        result = torch.pow(mean2, 1 / p_tensor) 

        return result
    
    def schrodinger_solution(self,f,tx):
        t = tx[:,0][::self.N]
        x = tx[:,1][:self.N]
        
        f_grid = f(x)
        f_grid = torch.complex(f_grid[:,0],f_grid[:,1])

        f_hat = torch.fft.fft(f_grid)

        xi = 2*torch.pi*torch.fft.fftfreq(x.size(0), d=(x[1].item() - x[0].item()))
        xi = xi.to(x.device)

        phases = torch.exp(-1j * t[:,None] * xi[None, :]**2)

        u_hat = f_hat[None,:] * phases

        u_grid = torch.fft.ifft(u_hat)
        return u_grid.flatten()
    
    
    def calc_loss(self):
        def diff(x):
            x0 = self.set.initial_data(x)
            return self.initial_condition(x)-self.net(x0)

        t = self.tx[:,0]
        x = self.tx[:,1]
        
        if self.model == 1:
            u = self.schrodinger_solution(diff,self.tx)
            linear_evol = torch.zeros((self.N * self.M, 2), dtype=torch.float64)
            linear_evol[:, 0] = u.real
            linear_evol[:, 1] = u.imag
            loss1 = self.J_pq(linear_evol,self.p*torch.ones(1),self.q*torch.ones(1),self.N,self.M,self.weights)[0]

            sol_pde = self.pde(self.net(self.txpr),self.txpr)
            loss2 = self.J_pq(sol_pde,self.ppr*torch.ones(1),self.qpr*torch.ones(1),self.Npr,self.Mpr,self.weightspr)[0]
        
        elif self.model == 2:
            difference = diff(x)
            sol_pde = self.pde(self.net(self.txpr),self.txpr)
            loss1 = torch.mean(torch.norm(difference,dim=1)**2)
            loss2 = torch.mean(torch.norm(sol_pde,dim=1)**2)

        x0 = self.set.initial_data(x)
        diff_out_init = diff(x)
        tildeA = self.J_H1(diff_out_init,x,torch.ones(x.size(0)))

        A = self.J_inftyH1(self.net(self.tx),self.tx,self.N,self.M,self.weights)
        B = self.J_pq(self.net(self.tx),self.p*torch.ones(1),self.q*torch.ones(1),self.N,self.M,self.weights)[0]
    
        return loss1 + self.gamma * loss2, tildeA, A, B 
    
    def train(self,iters):
        global iter_count
        iter_count = 0
        def closure():
            global iter_count
            self.optimizer.zero_grad()
            loss,tildeA,A,B = self.calc_loss() 
            loss.backward() 
            
            self.losses.append(loss.item())
            self.tildeA.append(tildeA.item())
            self.A.append(A.item())
            self.B.append(B.item())
            
            iter_count += 1
            if iter_count % 100 == 0:
                test_u = self.u_true(self.test)
                difference = torch.stack([test_u.real,test_u.imag],dim=1) - self.net(self.test)
                LpW1q_error = self.J_pq(difference,self.p*torch.ones(1),self.q*torch.ones(1),self.n_test,self.m_test,torch.ones((self.m_test,self.n_test)))[0].item()
                LinftyH1_error = self.J_inftyH1(difference,self.test,self.n_test,self.m_test,torch.ones((self.m_test,self.n_test))).item()
                
                qq = torch.linspace(2,40,77)
                pp = 2*torch.pow(1/2-1/qq,-1)
                
                norm_vector = self.J_pq(difference,pp[1:],qq[1:],self.n_test,self.m_test,torch.ones((self.m_test,self.n_test)))
                norm_vector = torch.cat((torch.Tensor([LinftyH1_error]),norm_vector))
                Sprime_error = torch.max(norm_vector).item()
                
                self.LpW1q_norms.append(LpW1q_error)
                self.LinftyH1_norms.append(LinftyH1_error)
                self.Sprime_norms.append(Sprime_error)
                
                time_now = datetime.now() - start_time
                print(f'Iterations {iter_count} || loss: {self.losses[-1]:.7f}  || Ã: {self.tildeA[-1]:.7f}  || A: {self.A[-1]:.3f} || B: {self.B[-1]:.3f} || LpW1q: {LpW1q_error:.7f}|| LinfH1: {LinftyH1_error:.7f}|| Spr: {Sprime_error:.7f} || ETA: {time_now.total_seconds():.3f}')
                print('-'*30)
            return loss
        
        start_time = datetime.now()
        try:
            for _ in range(iters):
                self.optimizer.step(closure)
        except KeyboardInterrupt:
            pass

        final_time = datetime.now() - start_time
        self.eta = final_time.total_seconds()
        print("Elapsed Time=",self.eta,"[s]")
        
    
    
    def plot2D(self,sol,T):
        N_test = 200
        X_test = torch.linspace(self.set.lb_x, self.set.ub_x, N_test)
        
        fig1, ax1 = plt.subplots(figsize=(9,7))
        fig2, ax2 = plt.subplots(figsize=(9,7))
        colors_exact = ['tomato','skyblue','yellowgreen']
        colors_dnn = ['black','grey','indigo']
        for i in range(len(T)):
            Xt = torch.stack((torch.ones(N_test)*T[i],X_test))
            Xt = Xt.T
            y_test = self.net(Xt).squeeze()
            u = sol(Xt)
            u = torch.stack((u.real,u.imag))
            y_true = u.T
            
            ax1.plot(X_test, y_true.detach()[:,0],color = colors_exact[i],label = '$u(t=$' + str(T[i]) + '$,\\cdot)$') 
            line1, = ax1.plot(X_test, y_test.detach()[:,0],'--',color= colors_dnn[i],label = '$u_{DNN,\\#}(t=$' + str(T[i]) + '$,\\cdot)$') 
            line1.set_dashes([7,6])
            
            ax2.plot(X_test, y_true.detach()[:,1],color = colors_exact[i],label = '$u(t=$' + str(T[i]) + '$,\\cdot)$')  
            line2, = ax2.plot(X_test, y_test.detach()[:,1],'--',color= colors_dnn[i],label = '$u_{DNN,\\#}(t=$' + str(T[i]) + '$,\\cdot)$') 
            line2.set_dashes([7,6])
            
        ax1.set_title('Re$(u)$')
        ax1.legend(loc='best',handlelength=3)
        ax1.grid()
        ax2.set_title('Im$(u)$')
        ax2.legend(loc='best',handlelength=3)
        ax2.grid()
        
        plt.show()
        fig1.savefig('2DReal-T'+str(len(T))+'.png', dpi=300, bbox_inches='tight') 
        fig2.savefig('2DImag-T'+str(len(T))+'.png', dpi=300, bbox_inches='tight') 
        
    def plot(self,tipo,plot_log = True):

        if tipo == '3D':
            %matplotlib inline
            fig = plt.figure(figsize=(10,7))
            ax = fig.add_subplot(2, 2, 1)
            yline = np.linspace(self.set.lb_I, self.set.ub_I, 1000)
            xline = np.linspace(self.set.lb_x, self.set.ub_x, 1000)
            X, Y = np.meshgrid(xline, yline)
            positions = np.column_stack([Y.ravel(), X.ravel()])
            U1 = self.net(torch.Tensor(positions))
            mini1 = torch.min(U1[:,0])
            maxi1 = torch.max(U1[:,0])
            UU1 = U1[:,0].view((len(xline),len(yline)))
            levels = np.linspace(mini1.item(), maxi1.item(), 50)
            surf = ax.contourf(Y, X, UU1.detach(),cmap='hot', levels = levels)
            ax.set_title('Re$(u_{DNN,\\#})$')
            ax.set_ylabel('$x$')
            fig.colorbar(surf, ticks = np.linspace(mini1.item(),maxi1.item(),10))

            ax = fig.add_subplot(2, 2, 2)
            U2 = self.sol(torch.Tensor(positions))
            UU2 = U2.real.view((len(xline),len(yline)))
            mini2 = torch.min(UU2)
            maxi2 = torch.max(UU2)
            levels = np.linspace(mini2.item(), maxi2.item(), 50)
            surf = ax.contourf(Y, X, UU2.detach(),cmap='hot', levels = levels)
            ax.set_title('Re$(u)$')
            fig.colorbar(surf, ticks = np.linspace(mini2.item(),maxi2.item(),10))
            
            ax = fig.add_subplot(2, 2, 3)
            
            
            mini3 = torch.min(U1[:,1])
            maxi3 = torch.max(U1[:,1])
            UU3 = U1[:,1].view((len(xline),len(yline)))
            levels = np.linspace(mini3.item(), maxi3.item(), 50)
            surf = ax.contourf(Y, X, UU3.detach(),cmap='hot', levels = levels)
            ax.set_title('Im$(u_{DNN,\\#})$')
            ax.set_xlabel('$t$')
            ax.set_ylabel('$x$')
            fig.colorbar(surf, ticks = np.linspace(mini3.item(),maxi3.item(),10))

            ax = fig.add_subplot(2, 2, 4)
            UU4 = U2.imag.view((len(xline),len(yline)))
            mini4 = torch.min(UU4)
            maxi4 = torch.max(UU4)
            levels = np.linspace(mini4.item(), maxi4.item(), 50)
            surf = ax.contourf(Y, X, UU4.detach(),cmap='hot',levels = levels)
            ax.set_title('Im$(u)$')
            ax.set_xlabel('$t$')
            fig.colorbar(surf, ticks = np.linspace(mini4.item(),maxi4.item(),10))
            plt.show()
            fig.savefig('3D.png', dpi=300, bbox_inches='tight') 
        
        elif tipo == 'absolute value':
            %matplotlib inline
            fig = plt.figure(figsize=(10,7))
            ax = fig.add_subplot(2, 1, 1)

            yline = np.linspace(self.set.lb_I, self.set.ub_I, 1000)
            xline = np.linspace(self.set.lb_x, self.set.ub_x, 1000)
            X, Y = np.meshgrid(xline, yline)
            positions = np.column_stack([Y.ravel(), X.ravel()])
            U1 = self.net(torch.Tensor(positions))
            UU1 = torch.norm(U1,dim=1)
            mini1 = torch.min(UU1)
            maxi1 = torch.max(UU1)
            UU1 = UU1.view((len(xline),len(yline)))
            U2 = self.sol(torch.Tensor(positions))
            UU2 = torch.abs(U2)
            mini2 = torch.min(UU2)
            maxi2 = torch.max(UU2)
            UU2 = UU2.view((len(xline),len(yline)))
            
            
            levels = np.linspace(mini1.item(), maxi1.item(), 50)
            surf = ax.contourf(Y, X, UU1.detach(),cmap='hot', antialiased=False, levels = levels)
            fig.colorbar(surf, ticks = np.linspace(mini1.item(),maxi1.item(),10))
            
            ax.set_ylabel('$x$')
            ax.set_title('$|u_{DNN,\\#}|$')
            ax = fig.add_subplot(2, 1, 2)
            
            
            levels = np.linspace(mini2.item(), maxi2.item(), 50)
            surf = ax.contourf(Y, X, UU2.detach(),cmap='hot', antialiased=False, levels = levels)
            
            fig.colorbar(surf, ticks = np.linspace(mini2.item(),maxi2.item(),10))
            
            ax.set_xlabel('$t$')
            ax.set_ylabel('$x$')
            ax.set_title('$|u|$')
            
            plt.show()
            fig.savefig('Absolute value.png', dpi=300, bbox_inches='tight') 

        elif tipo == 'absolute difference':
            %matplotlib inline
            fig = plt.figure(figsize=(10,7))
            ax = fig.add_subplot(1, 1, 1)

            yline = np.linspace(self.set.lb_I, self.set.ub_I, 1000)
            xline = np.linspace(self.set.lb_x, self.set.ub_x, 1000)
            X, Y = np.meshgrid(xline, yline)
            positions = np.column_stack([Y.ravel(), X.ravel()])
            U1 = self.net(torch.Tensor(positions))
            U1 = torch.complex(U1[:,0],U1[:,1])
            U2 = self.sol(torch.Tensor(positions))
            UU2 = torch.abs(U1-U2)
            mini = torch.min(UU2)
            maxi = torch.max(UU2)
            UU2 = UU2.view((len(xline),len(yline)))
            


            levels = np.linspace(mini.item(),maxi.item(),50)
            surf = ax.contourf(Y, X, UU2.detach(),cmap='hot', antialiased=False, levels = levels)
            fig.colorbar(surf, ticks = np.linspace(mini.item(),maxi.item(),10))
            
            ax.set_xlabel('$t$')
            ax.set_ylabel('$x$')
            ax.set_title('$|u_{DNN,\\#}-u|$')
            plt.show()  
            fig.savefig('Difference.png', dpi=300, bbox_inches='tight') 

        elif tipo == 'tilde A':
            %matplotlib inline

            fig, ax = plt.subplots(1,1,figsize=(9.5,5))
            if plot_log:
                ax.loglog(range(len(self.tildeA)),self.tildeA)
            else:
                ax.plot(range(len(self.tildeA)),self.tildeA)
            ax.grid()
            ax.set_title('$\\tilde A$')
            ax.set_xlabel('Iterations')
            plt.show()
            fig.savefig('tildeA.png', dpi=300, bbox_inches='tight') 

        elif tipo == 'A':
            %matplotlib inline

            fig, ax = plt.subplots(1,1,figsize=(9.5,5))
            if plot_log:
                ax.loglog(range(len(self.A)),self.A)
            else:
                ax.plot(range(len(self.A)),self.A)
            ax.grid()
            ax.set_title('$A$')
            ax.set_xlabel('Iterations')
            plt.show()
            fig.savefig('A.png', dpi=300, bbox_inches='tight') 

        elif tipo == 'B':
            %matplotlib inline

            fig, ax = plt.subplots(1,1,figsize=(9.5,5))
            if plot_log:
                ax.loglog(range(len(self.B)),self.B)
            else:
                ax.plot(range(len(self.B)),self.B)
            ax.grid()
            ax.set_title('$B$')
            ax.set_xlabel('Iterations')
            plt.show()
            fig.savefig('B.png', dpi=300, bbox_inches='tight') 

        elif tipo == 'loss':
            %matplotlib inline

            fig, ax = plt.subplots(1,1,figsize=(9.5,5))
            if plot_log:
                ax.loglog(range(len(self.losses)),self.losses)
            else:
                ax.plot(range(len(self.losses)),self.losses)
            ax.grid()
            ax.set_title('Loss')
            ax.set_xlabel('Iterations')
            plt.show()
            fig.savefig('loss.png', dpi=300, bbox_inches='tight') 

In [ ]:
class PINN_study(object):
    def __init__(self, PINN, Solution, n_test, m_test):
        self.PINN = PINN
        self.Solution = Solution
        self.n_test = n_test
        self.m_test = m_test
        
        self.test_data = PINN.set.generate_data(n_test,m_test)
        self.test_u = Solution(self.test_data)
        self.difference = torch.stack([self.test_u.real,self.test_u.imag],dim=1) - PINN.net(self.test_data)
        
    def LpW1q_norm(self):
        return self.PINN.J_pq(self.difference,self.PINN.p*torch.ones(1),self.PINN.q*torch.ones(1),self.n_test,self.m_test,torch.ones((self.m_test,self.n_test)))[0]
    
    def LinftyH1_norm(self):
        return self.PINN.J_inftyH1(self.difference,self.test_data,self.n_test,self.m_test,torch.ones((self.m_test,self.n_test)))
    
    def Sprime_norm(self,lb_q,ub_q,n_q):
        qq = torch.linspace(lb_q,ub_q,n_q)
        pp = 2*torch.pow(1/2-1/qq,-1)
        aux = self.PINN.J_inftyH1(self.difference,self.test_data,self.n_test,self.m_test,torch.ones((self.m_test,self.n_test)))
        
        norm_vector = self.PINN.J_pq(self.difference,pp[1:],qq[1:],self.n_test,self.m_test,torch.ones((self.m_test,self.n_test)))
        norm_vector = torch.cat((torch.Tensor([aux.item()]),norm_vector))
        return norm_vector

In [ ]:
alpha = 3
lb_I = -1
ub_I = 1
lb_x = -8
ub_x = 8
#S1 = PeregrineBreather()
#S1 = KuznetsovMaBreather(a=2)
#S1 = AkhmedievBreather(a=1/8)
S1 = Soliton(c = 3,nu = 3)
#S1 = StandingWave(omega = 1,alpha = alpha)
pde = PDE(alpha)
SET = Set(lb_I,ub_I,lb_x,ub_x)
N = 32
Npr = 32
M = 32
Mpr = 32
n_layers = 5
wide = 20
lr = 1
max_iter = 3000

In [ ]:
P1 = PINN(S1.initial_condition,pde,SET,N,Npr,M,Mpr,
          n_layers,wide,lr,max_iter,u_true = S1,activation = Sin,
          optimizer = 'LBFGS',model=1,conv=False,Drop = 0, gamma = 1)

In [ ]:
P1.train(1)

In [ ]:
Study = PINN_study(P1, S1, 100, 100)
LpW1q = Study.LpW1q_norm()
LinfH1 = Study.LinftyH1_norm()
norm = Study.Sprime_norm(2,100,197)
Sprime = torch.max(norm)
print(LpW1q.item(),LinfH1.item(),Sprime.item())

In [ ]:
%matplotlib inline
qq = torch.linspace(2,100,197)
plt.plot(qq,norm.detach())
plt.grid()
plt.xlabel('q')
plt.ylabel("$L^p_tW_x^{1,q}$")
plt.title("Estimation of S' error")
plt.savefig('Estimation Sprime.png', dpi=300, bbox_inches='tight') 
plt.show()

In [ ]:
%matplotlib inline
P1.plot2D(S1,[-0.5,0.1,0.3])

In [ ]:
%matplotlib inline
P1.plot2D(S1,[P1.set.lb_I,0,P1.set.ub_I])

In [ ]:
P1.plot('tilde A')

In [ ]:
P1.plot('A',plot_log = False)

In [ ]:
P1.plot('B',plot_log = False)

In [ ]:
P1.plot('loss')

In [ ]:
P1.plot('3D',S1)

In [ ]:
P1.plot('absolute value',S1)

In [ ]:
P1.plot('absolute difference',S1)

In [ ]:
%matplotlib inline

plt.figure(figsize=(6,4))
plt.loglog(range(0,len(P1.LpW1q_norms)*100,100),P1.LpW1q_norms)
plt.title('$L_t^p W_x^{1,q}$ error')
plt.ylabel('Error')
plt.xlabel('Iterations')
plt.grid()
plt.savefig('LpW1q-error.png', dpi=300, bbox_inches='tight') 
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
plt.loglog(range(0,100*len(P1.LinftyH1_norms),100),P1.LinftyH1_norms)
plt.title('$L_t^{\\infty} H_x^{1}$ error')
plt.ylabel('Error')
plt.xlabel('Iterations')
plt.grid()
plt.savefig('LinfH1-error.png', dpi=300, bbox_inches='tight') 
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
plt.loglog(range(0,100*len(P1.Sprime_norms),100),P1.Sprime_norms)
plt.title('$S\'$ error')
plt.ylabel('Error')
plt.xlabel('Iterations')
plt.grid()
plt.savefig('Sprime-error.png', dpi=300, bbox_inches='tight') 
plt.show()

In [ ]:
'''Animation'''
from matplotlib.animation import FuncAnimation
plt.rcParams['animation.ffmpeg_path'] = '/opt/homebrew/bin/ffmpeg' 

M_test = 100
t = torch.linspace(P1.set.lb_I, P1.set.ub_I, M_test)

fig, ax = plt.subplots(1, 2, figsize=(9.5, 5))

N_test = 200
X_test = torch.linspace(P1.set.lb_x, P1.set.ub_x, N_test)

aux_t = torch.repeat_interleave(t, repeats=N_test)
aux_x = torch.tile(X_test, (M_test,))
tx_test = torch.stack((aux_t, aux_x), dim=1)

uu = S1(tx_test)

aux3 = torch.stack((torch.ones(N_test) * 0, X_test))
X0 = aux3.T
y_test = P1.net(X0).squeeze()
y_true = P1.initial_condition(X_test)

line_2, = ax[0].plot(X_test, y_true.detach()[:, 0], 'red', label=f'exact at time: {0:.2f}')
line_1, = ax[0].plot(X_test, y_test.detach()[:, 0], '--k', label=f'predicted at time: {0:.2f}')

ax[0].set_xlim(P1.set.lb_x, P1.set.ub_x)
ax[0].set_ylim(torch.min(uu.real), torch.max(uu.real))
ax[0].set_title('Re$(u)$')
ax[0].legend()
ax[0].grid()

line_4, = ax[1].plot(X_test, y_true.detach()[:, 1], 'red', label=f'exact at time: {0:.2f}')
line_3, = ax[1].plot(X_test, y_test.detach()[:, 1], '--k', label=f'predicted at time: {0:.2f}')

ax[1].set_xlim(P1.set.lb_x, P1.set.ub_x)
ax[1].set_ylim(torch.min(uu.imag), torch.max(uu.imag))
ax[1].set_title('Im$(u)$')
ax[1].legend()
ax[1].grid()

def update(frame):
    aux3 = torch.stack((torch.ones(N_test) * frame, X_test))
    X0 = aux3.T
    y_test = P1.net(X0).squeeze()
    aux4 = S1(X0)
    aux5 = torch.stack((aux4.real, aux4.imag))

    y_true = aux5.T
    line_2.set_ydata(y_true.detach()[:, 0])
    line_2.set_label(f'exact at time: {frame:.2f}')
    line_1.set_ydata(y_test.detach()[:, 0])
    line_1.set_label(f'predicted at time: {frame:.2f}')
    ax[0].legend()

    line_4.set_ydata(y_true.detach()[:, 1])
    line_4.set_label(f'exact at time: {frame:.2f}')
    line_3.set_ydata(y_test.detach()[:, 1])
    line_3.set_label(f'predicted at time: {frame:.2f}')
    ax[1].legend()

    return line_1, line_2

ani = FuncAnimation(fig, update, frames=t, blit=True)

ani.save('animation.mp4', writer='ffmpeg', fps=15) 
plt.show()


In [ ]:
'''Comparison: Finite Differences Method'''
def FDM(initial_cond,M,N,T,R):
    dt = 2*T/M
    dx = 2*R/N
    xline = torch.linspace(-R,R,N+1)
    tline = torch.linspace(-T,T,M+1)

    start_time = datetime.now()
    u = np.zeros((M+1,N+1),dtype=complex)
    psi = np.zeros((M+1,N+1),dtype=complex)
    aux = initial_cond(xline)
    psi[int(M/2),:] = torch.complex(aux[:,0],aux[:,1])
    u[int(M/2),:] = torch.complex(aux[:,0],aux[:,1])
    
    for m in range(int(M/2),M):
        for n in range(1,N):
            psi[m+1,n] = 2 * np.abs(u[m,n])**2 - psi[m,n]
            u[m+1,n] = u[m,n] + dt/dx**2*(u[m,n+1]-2*u[m,n]+u[m,n-1])*1j + 1j*dt*u[m,n] * psi[m+1,n]

    for m in range(int(M/2),0,-1):
        for n in range(1,N):
            psi[m-1,n] = 2 * np.abs(u[m,n])**2 - psi[m,n]
            u[m-1,n] = u[m,n] - dt/dx**2*(u[m,n+1]-2*u[m,n]+u[m,n-1])*1j - 1j*dt*u[m,n] * psi[m,n]
    final_time = datetime.now() - start_time
    print('FDM time: {}'.format(final_time.total_seconds()))
    return u



In [ ]:
#alpha = 3
#lb_I = -1
#ub_I = 1
#lb_x = -8
#ub_x = 8
#S1 = PeregrineBreather()
#S1 = KuznetsovMaBreather(a=0.75)
#S1 = AkhmedievBreather(a=1/8)
#S1 = Soliton(c = 3,nu = 5)
#S1 = StandingWave(omega = 5,alpha = alpha)

R = ub_x 
T = ub_I
N_FDM = 300
M_FDM = 180000
sol_FDM = FDM(S1.initial_condition,M_FDM,N_FDM,T,R)

In [ ]:
index = -1
index2 = 1

xline = torch.linspace(-R,R,N_FDM+1)
tline = torch.linspace(-T,T,M_FDM+1)
plt.figure(figsize=(20,15))
plt.plot(xline[index2:-index2],sol_FDM[index,index2:-index2].real,label='FDM')
tx = torch.zeros((N_FDM+1,2))
tx[:,0] = tline[index]*torch.ones(len(xline))
tx[:,1] = xline
uu = S1(tx)
plt.plot(xline[index2:-index2],uu.real[index2:-index2],label='Exact')
plt.plot(xline[index2:-index2],P1.net(tx).detach()[index2:-index2,0],label='DNN')
plt.grid()
plt.legend(loc='best')
plt.show()
plt.figure(figsize=(20,15))
plt.plot(xline[index2:-index2],sol_FDM[index,index2:-index2].imag,label='FDM')
plt.plot(xline[index2:-index2],uu.imag[index2:-index2],label='Exact')
plt.plot(xline[index2:-index2],P1.net(tx).detach()[index2:-index2,1],label='DNN')
plt.grid()
plt.legend(loc='best')
plt.show()
plt.figure(figsize=(10,10))
plt.plot(xline[index2:-index2],np.abs(sol_FDM[index,index2:-index2]-uu.numpy()[index2:-index2]),label='FDM-Exact')
plt.plot(xline[index2:-index2],torch.abs(P1.net(tx)[index2:-index2,0]+1j*P1.net(tx)[index2:-index2,1]-uu[index2:-index2]).detach(),label='DNN-Exact')
plt.grid()
plt.legend(loc='best')
plt.show()

In [ ]:
'''L^p_t W^{1,q}_x norm for FDM'''
index_x = 1
X, Y = np.meshgrid(xline, tline)
positions = np.column_stack([Y.ravel(), X.ravel()])
positions = torch.Tensor(positions)
U2 = S1(positions).view((len(tline),len(xline)))
UU2 = np.abs(sol_FDM-U2.numpy())
norms = UU2[:,index_x:-index_x]
inside = UU2[:,index_x:-index_x]**4

mean1 = np.mean(inside,axis=1)
inside2 = mean1**2

mean2 = np.mean(inside2)

print(mean2**(1/8))

In [ ]:
'''L^{\infty}H^1_x norm for FDM'''
dx = 2*R/N_FDM

dUU2= np.zeros_like(UU2)
dUU2[:, :-1] = (UU2[:, 1:] - UU2[:, :-1]) / dx
dUU2[:, -1] = (UU2[:, -1] - UU2[:, -2]) / dx 

mean1 = np.mean(UU2[:,index_x:-index_x]**2+dUU2[:,index_x:-index_x]**2,axis=1)
inside1 = mean1**(1/2)

maxx = np.max(inside1)

print(maxx)

In [ ]:
mini = np.min(UU2[:,index_x:-index_x])
maxi = np.max(UU2[:,index_x:-index_x])
levels = np.linspace(mini,maxi,50)
fig = plt.figure(figsize=(10,7))
ax = fig.add_subplot(1, 1, 1)
surf = ax.contourf(Y[:,index_x:-index_x], X[:,index_x:-index_x], UU2[:,index_x:-index_x],cmap='hot', antialiased=False, levels = levels)
ax.set_title('$|u_{FDM}-u|$')
ax.set_xlabel('t')
ax.set_ylabel('x')

fig.colorbar(surf, ticks = np.linspace(mini,maxi,10))

In [ ]:
P1.plot('absolute difference')